In [12]:
import cv2
import mediapipe as mp
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import plotly.graph_objects as go
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def list_files_in_directory(directory):

    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.isfile(file_path):
                file_paths.append(file_path)
    return file_paths

In [4]:
def get_folder_names(directory):

    folder_names = []

    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)

        if os.path.isdir(item_path):
            folder_names.append(item)

    return folder_names

In [9]:
folder_names = get_folder_names("data")
folder_names = [folder_names[2]]
folder_names

['chest fly machine']

In [10]:
directory = "output"

if not os.path.exists(directory):
    os.makedirs(directory)

In [11]:
marked_for_removal = []

for folder_name in folder_names:
    video_paths = list_files_in_directory(f'data/{folder_name}')
    all_clips = []
    print(f"Class: {folder_name}")
    for video_source in tqdm(video_paths):
        
        cap = cv2.VideoCapture(video_source)
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
                temp = []
                ret, frame = cap.read()

                if not ret:
                    break
                
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                
                results = pose.process(image)
            
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                
                try:
                    landmarks = results.pose_landmarks.landmark
                    for y in landmarks:
                        temp.extend([y.x,y.y,y.z,y.visibility])
                    
                except:
                    pass
                
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                        )               
                
                cv2.imshow('Mediapipe Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    marked_for_removal.append(video_source)
                    break
                else:
                    all_clips.append(temp)

            cap.release()
                
            cv2.destroyAllWindows()
 
    df = pd.DataFrame(all_clips)
    df["class"] = folder_name
    df.to_csv(f"output/{folder_name}.csv")

print(marked_for_removal)

Class: chest fly machine


  0%|          | 0/28 [00:00<?, ?it/s]

c:\Users\chand\anaconda3\envs\AllPackages\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
100%|██████████| 28/28 [03:49<00:00,  8.20s/it]


['data/chest fly machine\\chest fly machine_15.mp4', 'data/chest fly machine\\chest fly machine_16.mp4', 'data/chest fly machine\\chest fly machine_17.mp4', 'data/chest fly machine\\chest fly machine_18.mp4', 'data/chest fly machine\\chest fly machine_23.mp4', 'data/chest fly machine\\chest fly machine_24.mp4', 'data/chest fly machine\\chest fly machine_25.mp4', 'data/chest fly machine\\chest fly machine_26.mp4']


# Visualisation

In [13]:
df = pd.read_csv('output\\barbell biceps curl.csv')